In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:
data_bt_usd = pd.read_csv("Bitcoin.csv")
data_bt_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   396 non-null    object
 1   Şimdi   396 non-null    object
 2   Açılış  396 non-null    object
 3   Yüksek  396 non-null    object
 4   Düşük   396 non-null    object
 5   Hac.    396 non-null    object
 6   Fark %  396 non-null    object
dtypes: object(7)
memory usage: 21.8+ KB


In [ ]:
df = data_bt_usd.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [ ]:
df.head(10)

,Tarih,Şimdi
0,15.12.2022,"17.672,1"
1,14.12.2022,"17.796,4"
2,13.12.2022,"17.778,6"
3,12.12.2022,"17.210,4"
4,11.12.2022,"17.093,3"
5,10.12.2022,"17.127,2"
6,09.12.2022,"17.125,7"
7,08.12.2022,"17.225,7"
8,07.12.2022,"16.835,2"
9,06.12.2022,"17.089,3"


In [ ]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '')))
df

,Tarih,Şimdi
0,15.12.2022,17.6721
1,14.12.2022,17.7964
2,13.12.2022,17.7786
3,12.12.2022,17.2104
4,11.12.2022,17.0933
...,...,...
391,19.11.2021,58.0834
392,18.11.2021,56.9553
393,17.11.2021,60.3678
394,16.11.2021,60.0891


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tarih   396 non-null    object 
 1   Şimdi   396 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.3+ KB


In [ ]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			 break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat*1000)

Epoch 1/200
13/13 [==============================] - 2s 37ms/step - loss: 319.2747
Epoch 2/200
13/13 [==============================] - 1s 60ms/step - loss: 43.9222
Epoch 3/200
13/13 [==============================] - 1s 72ms/step - loss: 11.5577
Epoch 4/200
13/13 [==============================] - 1s 71ms/step - loss: 4.5538
Epoch 5/200
13/13 [==============================] - 0s 37ms/step - loss: 3.6096
Epoch 6/200
13/13 [==============================] - 0s 35ms/step - loss: 3.2956
Epoch 7/200
13/13 [==============================] - 0s 36ms/step - loss: 3.3020
Epoch 8/200
13/13 [==============================] - 0s 35ms/step - loss: 3.3080
Epoch 9/200
13/13 [==============================] - 0s 36ms/step - loss: 3.2013
Epoch 10/200
13/13 [==============================] - 0s 36ms/step - loss: 3.0826
Epoch 11/200
13/13 [==============================] - 0s 37ms/step - loss: 3.2819
Epoch 12/200
13/13 [==============================] - 0s 35ms/step - loss: 3.2012
Epoch 13/200
13/13 [=

In [ ]:
print(f"Prediction: {round(yhat[0][0]*1000)} $ Bitcoin/USD") # onluk olarak yukarı yuvarlandı.

Prediction: 17658 $ Bitcoin/USD
